### 아래 내용은 글로벌 기업 (주)인텔에서 개발한 <br> Intel® AI for Youth Program 내용을 
대한민국 Implementation Partner사인 (주)Brain AI와 Brain AI Lead Coach Network에서 우리나라 초, 중, 고 학생들의 AI 교육을 위해 현지화 한 내용입니다. <br>
(주)Brain AI와 NDA를 체결한 학교에서만 사용 가능하며 NDA를 준수해야합니다. <br>
상업적 사용은 불가하며, 학교내에서 학생들 교육활동에 자유롭게 사용가능합니다.

## 프로젝트 제목: BrainAI 얼굴 인식 AI 시스템 만들기
### Intel OpenVINO Pre-trained 모델을 활용하여 얼굴 인식 AI 시스템 개발 실습

## 1. Open-Model-Zoo 모델 다운로드

### Task 1. 사용할 모델 문서를 참조하고 모델의 입력 및 출력을 알아봅시다.

얼굴 인식 모델(face-detection-adas-0001)에 대해서 [여기](https://docs.openvino.ai/2022.1/omz_models_model_face_detection_adas_0001.html)에서 알아봅니다.

<img src="https://docs.openvino.ai/2022.1/_images/face-detection-adas-0001.png" style="width:400px; float:left;" />
<div style="clear:both;"></div>

네트워크(모델)는 모양이 [1, 1, N, 7] 인 blob을 출력합니다. <br>
여기서 N은 감지 된 경계 상자의 수입니다. 각 탐지에 대해 설명은 다음과 같은 형식을 갖습니다.<br>
<b>[image_id, label, conf, x_min, y_min, x_max, y_max]

A very basic introduction to using face detection models with OpenVINO™ 

The [Model/face-detection-adas-0001](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/face-detection-adas-0001/README.md) model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/) is used. It detects faces in images and returns a blob of data 

with shape: `1, 1, 200, 7` in the format `1, 1, N, 7`, where `N` is the number of detected
bounding boxes. The results are sorted by confidence in decreasing order. Each detection has the format
[`image_id`, `label`, `conf`, `x_min`, `y_min`, `x_max`, `y_max`], where:

- `image_id` - ID of the image in the batch
- `label` - predicted class ID (1 - face)
- `conf` - confidence for the predicted class
- (`x_min`, `y_min`) - coordinates of the top left bounding box corner
- (`x_max`, `y_max`) - coordinates of the bottom right bounding box corner

### Task 2. 사용할 모델을 omz_downloader를 사용하여 다운로드해 봅시다.

#### 2.1 omz_downloader 옵션 확인하기

In [ ]:
! omz_downloader -h

#### 2.2 사용가능한 모든 models 출력

In [ ]:
! omz_downloader --print_all

#### 2.3 'face detection for ADAS' model 다운로드

In [ ]:
! omz_downloader --name face-detection-adas-0001 --precision FP16

#### 2.4 다운로드한 모델을 model 폴더로 복사

'Intel'폴더에 다운로드한 모델 파일 'face-detection-adas-0001.bin', 'face-detection-adas-0001.xml' 을 프로젝트 폴더 안의 'models' 폴더로 복사합니다.

<img src="images/file_copy.jpg" style="width:400px; float:left;" />
<div style="clear:both;"></div>

### Task 3. 'face-detection-adas-0001' 모델을 사용하여 얼굴 인식을 해봅시다.

#### 3-1. 사용할 OpenVino Model Zoo 불러와서 입출력값 확인하기  

In [19]:
import openvino as ov

In [20]:
core = ov.Core()

model = core.read_model(model="models/face-detection-adas-0001.xml")
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)
print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


#### 3-2. 사용할 이미지(데이터) 불러와서 전처리하기

In [21]:
import cv2
import numpy as np

frame = cv2.imread("images/test.jpeg")

In [22]:
resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

#### 3-3. 추론(Inference)

In [23]:
face_output = face_model([input_frame])[face_output_layer]

#### 3-4. 배포(Deployment)

In [24]:
#추론한 값(output), 이미지(image), 임계값(conf)을 가져와 
#이미지에 얼굴에 박스를 그리는 DrawBoundingBoxes함수
def DrawBoundingBoxes(output, image, conf):

    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기

    return canvas

In [17]:
canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)

cv2.imshow("Deployment", canvas)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Task 4. 배경이미지에 결과 출력하기

In [25]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 300
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [26]:
background = "./images/Background.jpg"  #사용할 배경화면 경로
canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)  
bg = cv2.imread(background)

deployment = AddBackground(canvas, bg)
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Task 5. 영상(Webcam) 입력받아 얼굴 인식

In [14]:
#camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
camera = cv2.VideoCapture('./images/people.mp4')
background = "./images/Background.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]

    canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)
    deployment = AddBackground(canvas, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()

### Task 6. Gradio 사용하여 결과 출력하기

#### 6-1. 웹에서 이미지 입력받아 결과 출력하기

In [ ]:
def input_images(input_img):
    canvas = DrawBoundingBoxes(face_output, input_img, conf=0.5)
    return canvas

In [ ]:
import gradio as gr

demo = gr.Interface(input_images, gr.Image(), "image")
demo.launch()

#### 6-2. 웹에서 캠으로 영상받아서 결과 출력하기

In [ ]:
import gradio as gr
import numpy as np

camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./images/Background.jpg"
bg = cv2.imread(background)

def flip(input_image):
    canvas = DrawBoundingBoxes(face_output, np.flip(input_image, 1), conf=0.5)
    #canvas = DrawBoundingBoxes(face_output, input_image, conf=0.5)
    return AddBackground(canvas, bg)
''''''''''''''''''''''''
#deployment = AddBackground(canvas, bg)

demo = gr.Interface(flip, gr.Image(sources=["webcam"], streaming=True), "image", live=True )

demo.launch()

### 추가 미션. Hello Bot 만들어보기

In [ ]:
#소리를 출력하기 위해  playsound 라이브러리 설치 <미설치시 사용>
#!pip install playsound==1.2.2   

In [ ]:
from playsound import playsound

file_path = './sound/hello.mp3'   #소리 파일 경로 지정
 
def sayHello():       # 설정한 경로에서 실행할 파일 스피커로 출력하는 함수
    playsound(file_path)

In [ ]:
def DrawBoundingBoxes(output, image, conf):

    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
        
        detect = 1   # 얼굴인식 여부 1:인식, 0: 미인식
    return canvas, detect

In [ ]:
detect_check = 0   #얼굴 인식 여부 확인 변수(0:미인식, 1:인식)
frame = cv2.imread("images/test.jpeg")
canvas, detect_check = DrawBoundingBoxes(face_output, frame, conf=0.5)
 
cv2.imshow("Person", canvas)

if detect_check == 1:  #얼굴 인식되었으면 소리 출력
    sayHello()
    
cv2.waitKey(0)
cv2.destroyAllWindows()